# Train Model

In [1]:
from comet_ml import Experiment
from pytorch_lightning.loggers import CometLogger
import datasets
import torch
from architectures import *
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
import torchvision
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import LearningRateMonitor
import optuna
import logging
%config Completer.use_jedi = False

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

GeForce RTX 2070
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [2]:
optuna.logging.enable_default_handler()
logging.getLogger().setLevel(logging.INFO)  # Setup the root logger.
logging.getLogger().addHandler(logging.FileHandler('optuna.log'))
optuna.logging.enable_propagation() 

In [3]:
# Configure Dataset
dataset = datasets.SpecShadOcc("SpecShadOcc/", 9)
test_length = 20
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset)-test_length,test_length])
train_datasets = [datasets.SpecShadOccN(dataset, i) for i in range(1,9)]
train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True)
#train_loaders = [DataLoader(train_dataset, batch_size=2, shuffle=True) for train_dataset in train_datasets] 
test_loader = DataLoader(test_dataset,batch_size=10)

def objective(trial: optuna.trial.Trial):

    planes  = trial.suggest_int("planes", low=10, high=20, step=1)
    downsampling_factor= trial.suggest_int("downsampling_factor", low=0, high=3, step=1)
    encoder_num_blocks = trial.suggest_int("encoder_num_blocks", low=5, high=20, step=5)
    decoder_num_blocks = trial.suggest_int("decoder_num_blocks", low=5, high=20, step=5)
    smooth_num_blocks  = trial.suggest_int("smooth_num_blocks", low=5, high=20, step=5)

    agg_block = trial.suggest_categorical("agg_block",["Attention"])
    
    if agg_block=="Attention":
        agg_params = { 
            "mode": trial.suggest_categorical("mode",["softmax","sum","mean"]),
            "num_heads": trial.suggest_int("num_heads", low=4, high=8, step=4),
            "dim_feedforward": trial.suggest_int("dim_feedforward", low=64, high=128, step=64),
            "num_layers": trial.suggest_int("num_layers", low=2, high=4, step=1),
        }
    else:
        agg_params = {}
    
    #lr_monitor = LearningRateMonitor(logging_interval='step')
    try:
        net = DeepAggNet(planes=planes, downsampling_factor=downsampling_factor,
                         encoder_num_blocks=encoder_num_blocks, 
                         decoder_num_blocks=decoder_num_blocks, smooth_num_blocks=smooth_num_blocks,
                         agg_block=agg_block, **agg_params)

        # Create Logger
        comet_logger = CometLogger(
            api_key="tMEjeyq5M7v1IPRCvS5fyGyuo",
            workspace="semjon", # Optional
            project_name="specshadocc", # Optional
            # rest_api_key=os.environ["COMET_REST_KEY"], 
            save_dir='./hyperparameter',
            experiment_name="specshadocc-" + agg_block, # Optional,
            display_summary_level = 0
        )

        early_stop = pl.callbacks.EarlyStopping('val_loss', patience=5)
        trainer = pl.Trainer(gpus=1,logger=comet_logger, max_epochs=100, 
                             #fast_dev_run=True,
                             callbacks=[early_stop], 
                             progress_bar_refresh_rate=0)
        trainer.fit(net, train_loader, val_dataloaders=test_loader)
        return trainer.test(net, test_loader)[0]["test_loss"]
    except:
        print("An exception occurred")
        return 0.1
    


In [4]:
sampler = optuna.samplers.TPESampler(seed=10)

study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, n_trials=100)

[I 2021-03-19 14:42:06,166] A new study created in memory with name: no-name-7cb6c7d1-cc71-4bed-bee5-4399f9e3f748
[I 2021-03-19 14:42:06,176] Trial 0 finished with value: 0.1 and parameters: {'planes': 15, 'downsampling_factor': 1, 'encoder_num_blocks': 5, 'decoder_num_blocks': 20, 'smooth_num_blocks': 5, 'agg_block': 'Attention', 'mode': 'sum', 'num_heads': 8, 'dim_feedforward': 64, 'num_layers': 3}. Best is trial 0 with value: 0.1.
CometLogger will be initialized in online mode
GPU available: True, used: True
TPU available: None, using: 0 TPU cores


An exception occurred


COMET INFO: Experiment is live on comet.ml https://www.comet.ml/semjon/specshadocc/f2014bcee9bc4622a01247dc8c2e59d7


  | Name       | Type                 | Params
----------------------------------------------------
0 | input      | Conv2d               | 280   
1 | output     | Conv2d               | 273   
2 | downsample | Sequential           | 1.8 K 
3 | upsample   | Sequential           | 1.8 K 
4 | encoder    | Sequential           | 72.4 K
5 | agg        | AttentionAggregation | 8.8 K 
6 | decoder    | Sequential           | 108 K 
7 | smooth     | Sequential           | 10.9 K
----------------------------------------------------
204 K     Trainable params
0         Non-trainable params
204 K     Total params
0.820     Total estimated model params size (MB)
[I 2021-03-19 14:42:13,970] Trial 1 finished with value: 0.1 and parameters: {'planes': 10, 'downsampling_factor': 1, 'encoder_num_blocks': 10, 'decoder_num_blocks': 15, 'smooth_num_blocks': 5, 'agg_block': 'Attention', 'mo

An exception occurred


COMET INFO: Experiment is live on comet.ml https://www.comet.ml/semjon/specshadocc/94b44e03888d4b3a8a597961f22c1b29


  | Name       | Type                 | Params
----------------------------------------------------
0 | input      | Conv2d               | 840   
1 | output     | Conv2d               | 813   
2 | downsample | Sequential           | 340 K 
3 | upsample   | Sequential           | 340 K 
4 | encoder    | Sequential           | 5.2 M 
5 | agg        | AttentionAggregation | 1.2 M 
6 | decoder    | Sequential           | 5.2 M 
7 | smooth     | Sequential           | 341 K 
----------------------------------------------------
12.6 M    Trainable params
0         Non-trainable params
12.6 M    Total params
50.294    Total estimated model params size (MB)
[I 2021-03-19 14:42:20,076] Trial 2 finished with value: 0.1 and parameters: {'planes': 30, 'downsampling_factor': 3, 'encoder_num_blocks': 5, 'decoder_num_blocks': 5, 'smooth_num_blocks': 20, 'agg_block': 'Attention', 'mod

...

An exception occurred


GPU available: True, used: True
TPU available: None, using: 0 TPU cores
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/semjon/specshadocc/47219d6bc1ad4ff69642dd7de9e7a4c5


  | Name       | Type                 | Params
----------------------------------------------------
0 | input      | Conv2d               | 700   
1 | output     | Conv2d               | 678   
2 | downsample | Sequential           | 236 K 
3 | upsample   | Sequential           | 236 K 
4 | encoder    | Sequential           | 10.8 M
5 | agg        | AttentionAggregation | 749 K 
6 | decoder    | Sequential           | 14.4 M
7 | smooth     | Sequential           | 67.8 K
----------------------------------------------------
26.5 M    Trainable params
0         Non-trainable params
26.5 M    Total params
106.024   Total estimated model params size (MB)
[I 2021-03-19 14:46:30,685] Trial 98 finished with value: 0.1 and parameters: {'planes': 25, 'downsampling_factor': 3, 'encoder_num_blocks': 15, 'decod

An exception occurred


COMET INFO: Experiment is live on comet.ml https://www.comet.ml/semjon/specshadocc/b6b40105b68740a6abe6c6e1b4800b00


  | Name       | Type                 | Params
----------------------------------------------------
0 | input      | Conv2d               | 700   
1 | output     | Conv2d               | 678   
2 | downsample | Sequential           | 236 K 
3 | upsample   | Sequential           | 236 K 
4 | encoder    | Sequential           | 10.8 M
5 | agg        | AttentionAggregation | 749 K 
6 | decoder    | Sequential           | 14.4 M
7 | smooth     | Sequential           | 67.8 K
----------------------------------------------------
26.5 M    Trainable params
0         Non-trainable params
26.5 M    Total params
106.024   Total estimated model params size (MB)
[I 2021-03-19 14:46:37,018] Trial 99 finished with value: 0.1 and parameters: {'planes': 25, 'downsampling_factor': 3, 'encoder_num_blocks': 15, 'decoder_num_blocks': 20, 'smooth_num_blocks': 5, 'agg_block': 'Attention', 'm

An exception occurred
